In [1]:
import pandas as pd
import numpy as np

In [2]:
pandemic = pd.read_csv("final/Pandemic_v1.csv")
prepandemic = pd.read_csv("final/prepandemic_v1.csv")
unemployment = pd.read_csv("final/Unemployment_v1.csv")

# Remove all Puerto Rico data
pandemic = pandemic.drop(pandemic[pandemic['stname'] == 'Puerto Rico'].index)

In [3]:
# Need state and county name
state = pandemic['stname']
county = pandemic['ctyname']

# The confirmed cases and death number is accumulated, used the last available date data
confirmed_cases = pandemic['confirmed_cases_20200418']
deaths = pandemic['deaths_20200418']

In [4]:
pandemic

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,emergency,k12,dayCare,nursing_visitorBan,...,deaths_20200409,deaths_20200410,deaths_20200411,deaths_20200412,deaths_20200413,deaths_20200414,deaths_20200415,deaths_20200416,deaths_20200417,deaths_20200418
0,1,1,Alabama,1,Autauga County,1001,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,1,1,1,1,1,1,1,1,2,2
1,2,1,Alabama,3,Baldwin County,1003,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,1,1,1,1,1,2,2,2,2,2
2,3,1,Alabama,5,Barbour County,1005,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0
3,4,1,Alabama,7,Bibb County,1007,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0
4,5,1,Alabama,9,Blount County,1009,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0
5,6,1,Alabama,11,Bullock County,1011,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0
6,7,1,Alabama,13,Butler County,1013,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0
7,8,1,Alabama,15,Calhoun County,1015,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,2,2
8,9,1,Alabama,17,Chambers County,1017,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,8,8,8,8,8,10,10,11,11,11
9,10,1,Alabama,19,Cherokee County,1019,2020-03-13,2020-03-18 00:00:00,2020-03-20 00:00:00,2020-03-19 00:00:00,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Get days between first confirmed case to now
import datetime
def days(pandemic_row):
    start = datetime.datetime(2020,1,22)
    end = datetime.datetime(2020,4,18)
    days = 0
    while start <= end:
        title = 'confirmed_cases_' +  start.strftime("%Y%m%d")
        if pandemic_row[title] == 0:
            days += 1
        else:
            break

        start += datetime.timedelta(days=1)
    
    return days
case_days = pandemic.apply(lambda x: days(x), axis = 1) 
case_days

0       62
1       53
2       72
3       68
4       63
5       64
6       63
7       56
8       57
9       63
10      63
11      64
12      71
13      63
14      63
15      64
16      63
17      71
18      64
19      65
20      65
21      59
22      71
23      63
24      64
25      52
26      66
27      64
28      68
29      61
        ..
3112    85
3113    56
3114    50
3115    64
3116    75
3117    52
3118    54
3119    63
3120    81
3121    58
3122    59
3123    66
3124    75
3125    52
3126    65
3127    80
3128    64
3129    55
3130    72
3131    59
3132    79
3133    56
3134    76
3135    50
3136    66
3137    63
3138    57
3139    71
3140    65
3141    88
Length: 3142, dtype: int64

In [14]:
# Use year 2018(closest) population density
populationdensity=prepandemic['popdensity_2018']
populationdensity_min = populationdensity.min()
populationdensity_mean = populationdensity.mean()
populationdensity_stage = populationdensity_mean - populationdensity_min
def populationdensity_category(populationdensity):
    if populationdensity < populationdensity_mean - populationdensity_stage * 0.9: 
        return '1_Very Low' 
    elif populationdensity < populationdensity_mean - populationdensity_stage * 0.3: 
        return '2_Low'  
    elif populationdensity < populationdensity_mean + populationdensity_stage * 0.3: 
        return '3_Medium'   
    elif populationdensity < populationdensity_mean + populationdensity_stage: 
        return '4_High' 
    else:
        return 'Very_High'
populationdensity_category = prepandemic.apply(lambda x: populationdensity_category(x['popdensity_2018']), axis = 1)

In [15]:
# Get top 3 most populations ethnicity group in the county
def ethnic_group_top(hispanic, white, black, indian, asian, hawaii_na, top):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[top - 1][0]

ethnic_group_top1 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 1), axis = 1)
ethnic_group_top2 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 2), axis = 1)
ethnic_group_top3 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 3), axis = 1)

In [16]:
# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  age 0-4 prop.
# column2:  age 5-9 prop.
# column3:  age 10-14 prop.
# column4:  age 15-19 prop.
# column5:  age 20-24 prop.
# column6:  age 25-29 prop.
# column7:  age 30-34 prop.
# column8:  age 35-39 prop.
# column9:  age 40-44 prop.
# column10: age 45-49 prop.
# column11: age 50-54 prop.
# column12: age 55-59 prop.
# column13: age 60-64 prop.
# column14: age 65-69 prop.
# column15: age 70-74 prop.
# column16: age 75-79 prop.
# column17: age 80-84 prop.
# column18: age 85+ prop.
def age_group_top(age, top):
    children_poportion = age[0] + age[1] + age[2] + age[3]
    young_adult_poportion = age[4] + age[5] + age[6] + age[7] + age[8]
    middle_adult_poportion = age[9] + age[10] + age[11] + age[12]
    senior_poportion = age[13] + age[14] + age[15] + age[16] + age[17]
    age_group_list = [('Children', children_poportion), ('Young Adult', young_adult_poportion), ('Middle Adult', middle_adult_poportion), ('Senior', senior_poportion)]
    age_group_list = sorted(age_group_list, key = lambda x: -x[1])
    return age_group_list[top - 1][0]
age_groups_top1 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 1), axis = 1)
age_groups_top2 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 2), axis = 1)
age_groups_top3 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 3), axis = 1)
age_groups_top4 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 4), axis = 1)

In [17]:
# Get all education adata
education = pd.read_excel("final/Education_By_County.xls")
education = education.drop(education[pd.isna(education['2013 Rural-urban Continuum Code'])].index)

In [18]:
states_map = pd.read_csv("final/states.csv")
states_map

for index, row in states_map.iterrows():
    pc = len(pandemic[pandemic['stname'] == row[1]])
    ec = len(education[education['State'] == row[0]])
    if pc != ec:
        print(f"In education data set[{row[0]}]: {ec} rows, in pandemic dataset[{row[1]}]: {pc} rows")  

In education data set[VA]: 134 rows, in pandemic dataset[Virginia]: 133 rows
In education data set[PR]: 78 rows, in pandemic dataset[Puerto Rico]: 0 rows


In [19]:
va_pandemic = pandemic[pandemic['stname'] == 'Virginia']
va_education = education[education['State'] == 'VA']
for index, row in va_education.iterrows():
    area = row['Area name']
    if len(va_pandemic[va_pandemic['ctyname'] == area]) == 0:
        print(f"{area} is missing in va_pandemic")

Bedford city is missing in va_pandemic


In [21]:
education = education.drop(education[education['Area name'] == 'Bedford city'].index)

# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  less than high school diploma poportion.
# column2:  high school diploma only poportion.
# column3:  some college or associate degress poportion.
# column4:  bachelor or higher education poportion.
def education_group_top(poportions, top):
    return 1
#     education_group = [('Less_than_high_school', poportions[0]), ('High_school', poportions[1]), ('Some_college_or_associate', poportions[2]), ('Bachelor_Or_Above', poportions[3])]
#     education_group = sorted(education_group, key = lambda x: -x[1])
#     return age_group_list[top - 1][0]

education_group_top1 = education.apply(lambda x: education_group_top(education.iloc[:, 9:13], 1), axis = 1)

In [22]:
populations = pd.read_excel("final/co-est2019-annres.xlsx")
populations['county'] = populations['Geographic Area'].apply(lambda x: x.split(',')[0][1:])
populations['state'] = populations['Geographic Area'].apply(lambda x: x.split(',')[1])
population = populations['Census']

In [35]:
result = pd.concat([state, county], axis=1)
                    
result['confirmed_cases'] = confirmed_cases
result['deaths'] = deaths
result['population'] = population
result['days'] = case_days
result['avearge_case'] = (100000.0) * confirmed_cases / case_days / population # Daily confirmed case per 100k people
result['populationdensity'] = populationdensity_category
result['toppest_ethnicity'] = ethnic_group_top1
result['second_ethnicity'] = ethnic_group_top2
result['third_ethnicity'] = ethnic_group_top3
result['toppest_age_group'] = age_groups_top1
result['second_age_group'] = age_groups_top2
result['third_age_group'] = age_groups_top3
result['fourth_age_group'] = age_groups_top4


In [36]:
result

,stname,ctyname,confirmed_cases,deaths,population,days,avearge_case,populationdensity,toppest_ethnicity,second_ethnicity,third_ethnicity,toppest_age_group,second_age_group,third_age_group,fourth_age_group
0,Alabama,Autauga County,25,2,54571,62,0.738901,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
1,Alabama,Baldwin County,109,2,182265,53,1.128359,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
2,Alabama,Barbour County,18,0,27457,72,0.910515,1_Very Low,Black,White,Hispanic,Young Adult,Middle Adult,Children,Senior
3,Alabama,Bibb County,26,0,22915,68,1.668571,1_Very Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
4,Alabama,Blount County,20,0,57322,63,0.553819,2_Low,White,Hispanic,Black,Young Adult,Middle Adult,Children,Senior
5,Alabama,Bullock County,9,0,10914,64,1.288483,1_Very Low,Black,White,Hispanic,Young Adult,Middle Adult,Children,Senior
6,Alabama,Butler County,13,0,20947,63,0.985101,1_Very Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
7,Alabama,Calhoun County,66,2,118572,56,0.993971,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
8,Alabama,Chambers County,240,11,34215,57,12.306083,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior
9,Alabama,Cherokee County,12,0,25989,63,0.732911,2_Low,White,Black,Hispanic,Middle Adult,Young Adult,Senior,Children
